### **Import Libraries**

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
%matplotlib inline
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.combine import SMOTETomek
import pickle

### **Import Dataset**

In [13]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
df = pd.read_csv(url, sep=';')
df.head(20)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
5,7.4,0.660,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5
6,7.9,0.600,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5
7,7.3,0.650,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7
8,7.8,0.580,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7
9,7.5,0.500,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5


### **Data Preprocessing**

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [4]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [5]:
df['quality'].value_counts()

quality
5    681
6    638
7    199
4     53
8     18
3     10
Name: count, dtype: int64

34 = bad
56 = middle
78 = good

### **Splitting Data**

In [6]:
X = df.drop('quality', axis=1)
y = df['quality']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### **Modeling**

In [8]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [9]:
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)

In [10]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00        10
           5       0.69      0.78      0.73       130
           6       0.60      0.64      0.62       132
           7       0.50      0.36      0.42        42
           8       0.00      0.00      0.00         5

    accuracy                           0.63       320
   macro avg       0.30      0.30      0.29       320
weighted avg       0.59      0.63      0.61       320



c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [14]:
print("\n--- Distribusi Kelas Awal ---")
print("Distribusi kelas pada data latih (sebelum resampling):")
print(sorted(Counter(y_train).items()))

# 4. Menerapkan SMOTE-Tomek untuk Menangani Imbalance
print("\nMenerapkan SMOTE-Tomek pada data latih...")
# Proses ini akan menyeimbangkan kelas minoritas (oversampling) dan membersihkan data (undersampling)
# Hanya diterapkan pada data latih (X_train, y_train)
smt = SMOTETomek(random_state=42, n_jobs=-1)
X_train_resampled, y_train_resampled = smt.fit_resample(X_train, y_train)
print("Proses resampling dengan SMOTE-Tomek selesai.")

print("\n--- Distribusi Kelas Setelah Resampling ---")
print("Distribusi kelas pada data latih (setelah resampling):")
print(sorted(Counter(y_train_resampled).items()))

# 5. Melatih Model Machine Learning pada Data yang Sudah Diseimbangkan
print("\nMemulai proses pelatihan model RandomForestClassifier pada data hasil resampling...")
# Kita melatih model menggunakan data latih yang sudah di-resampling
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight = 'balanced', n_jobs=-1)
model.fit(X_train_resampled, y_train_resampled)
print("Pelatihan model selesai.")

# 6. Mengevaluasi Performa Model
print("\n--- Evaluasi Model ---")
print("Evaluasi dilakukan pada data uji ASLI (unseen data) untuk mengukur performa generalisasi.")
# Melakukan prediksi pada data uji (X_test) yang tidak pernah dilihat model atau di-resampling
y_pred = model.predict(X_test)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi pada data uji (Test Accuracy): {accuracy:.4f}")

# Menampilkan laporan klasifikasi yang lebih detail
print("\nLaporan Klasifikasi:")
target_labels = sorted(y.unique())
print(classification_report(y_test, y_pred, target_names=[f'Kualitas {l}' for l in target_labels]))


--- Distribusi Kelas Awal ---
Distribusi kelas pada data latih (sebelum resampling):
[(3, 9), (4, 43), (5, 551), (6, 506), (7, 157), (8, 13)]

Menerapkan SMOTE-Tomek pada data latih...
Proses resampling dengan SMOTE-Tomek selesai.

--- Distribusi Kelas Setelah Resampling ---
Distribusi kelas pada data latih (setelah resampling):
[(3, 551), (4, 548), (5, 518), (6, 515), (7, 545), (8, 551)]

Memulai proses pelatihan model RandomForestClassifier pada data hasil resampling...
Pelatihan model selesai.

--- Evaluasi Model ---
Evaluasi dilakukan pada data uji ASLI (unseen data) untuk mengukur performa generalisasi.
Akurasi pada data uji (Test Accuracy): 0.6312

Laporan Klasifikasi:
              precision    recall  f1-score   support

  Kualitas 3       0.00      0.00      0.00         1
  Kualitas 4       0.25      0.60      0.35        10
  Kualitas 5       0.72      0.72      0.72       130
  Kualitas 6       0.69      0.56      0.62       132
  Kualitas 7       0.56      0.67      0.61 

In [12]:
# 7. Menyimpan Model ke File .pkl
model_filename = 'wine_model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(model, file)

print(f"\nModel telah berhasil dilatih dengan data seimbang dan disimpan sebagai '{model_filename}'.")


Model telah berhasil dilatih dengan data seimbang dan disimpan sebagai 'wine_model.pkl'.


In [15]:
# Simpan sebagai: buat_model_kategori.py

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

# 1. Memuat Dataset
try:
    df = pd.read_csv('wine_quality.csv')
    print("Dataset 'wine_quality.csv' berhasil dimuat.")
except FileNotFoundError:
    print("Error: File 'wine_quality.csv' tidak ditemukan.")
    exit()

# 2. Membuat Kolom Kategori Kualitas (TARGET BARU)
# Mendefinisikan batas untuk setiap kategori
# 0-4 -> Bad, 5-6 -> Middle, 7-8 -> Good
bins = [0, 4, 6, 8]
labels = ['Bad', 'Middle', 'Good']
df['quality_category'] = pd.cut(df['quality'], bins=bins, labels=labels, right=True)

# Membuang kolom 'quality' asli karena sudah tidak diperlukan
df = df.drop('quality', axis=1)

print("\nKolom target telah diubah menjadi kategori:")
print(df['quality_category'].value_counts())

# 3. Memisahkan Fitur (X) dan Target (y)
X = df.drop('quality_category', axis=1)
y = df['quality_category']

# 4. Membagi Dataset menjadi Data Latih dan Data Uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"\nDataset dibagi menjadi {len(X_train)} data latih dan {len(X_test)} data uji.")

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

# 5. Melatih Model Machine Learning
print("\nMemulai proses pelatihan model untuk memprediksi KATEGORI...")
# Menggunakan class_weight='balanced' untuk menangani imbalance antar kategori baru
model = RandomForestClassifier(
    n_estimators=200,          # Bisa coba ditambah untuk model yg lebih kuat
    random_state=42,
    class_weight='balanced',
    oob_score=True,
    n_jobs=-1
)
model.fit(X_train, y_train)
print("Pelatihan model selesai.")

# 6. Mengevaluasi Performa Model
print("\n--- Evaluasi Model ---")
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi pada data uji: {accuracy:.4f}")
print(f"Out-of-Bag (OOB) Score: {model.oob_score_:.4f}")

print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred))

Dataset 'wine_quality.csv' berhasil dimuat.

Kolom target telah diubah menjadi kategori:
quality_category
Middle    1319
Good       217
Bad         63
Name: count, dtype: int64

Dataset dibagi menjadi 1279 data latih dan 320 data uji.

Memulai proses pelatihan model untuk memprediksi KATEGORI...
Pelatihan model selesai.

--- Evaluasi Model ---
Akurasi pada data uji: 0.8375
Out-of-Bag (OOB) Score: 0.8686

Laporan Klasifikasi:
              precision    recall  f1-score   support

         Bad       0.00      0.00      0.00        13
        Good       0.58      0.33      0.42        43
      Middle       0.86      0.96      0.91       264

    accuracy                           0.84       320
   macro avg       0.48      0.43      0.44       320
weighted avg       0.79      0.84      0.80       320



c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [ ]:

# 7. Menyimpan Model ke File .pkl
# File ini akan berisi model yang memprediksi 'Bad', 'Middle', 'Good'
model_filename = 'wine_model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(model, file)

print(f"\nModel KATEGORI telah berhasil dilatih dan disimpan sebagai '{model_filename}'.")

## **FINAL PAKE INI**

In [16]:
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.combine import SMOTETomek
import pickle

print("Memulai eksekusi skrip...")

# 1. Memuat Dataset
try:
    df = pd.read_csv('wine_quality.csv')
    print("Dataset 'wine_quality.csv' berhasil dimuat.")
except FileNotFoundError:
    print("Error: File 'wine_quality.csv' tidak ditemukan.")
    exit()

# 2. Membuat Kolom Kategori Kualitas
bins = [0, 4, 6, 8]
labels = ['Bad', 'Middle', 'Good']
df['quality_category'] = pd.cut(df['quality'], bins=bins, labels=labels, right=True)
df = df.drop('quality', axis=1)

print("\nDistribusi kategori asli:")
print(df['quality_category'].value_counts())

# 3. Memisahkan Fitur (X) dan Target (y)
X = df.drop('quality_category', axis=1)
y = df['quality_category']

# 4. Membagi Dataset menjadi Data Latih dan Data Uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("\nDataset berhasil dibagi.")
print("Distribusi kategori pada data latih (sebelum resampling):")
print(sorted(Counter(y_train).items()))

# 5. Menerapkan SMOTE-Tomek untuk Menyeimbangkan Data Latih
print("\nMenerapkan SMOTE-Tomek pada data latih... Ini mungkin butuh beberapa saat.")
# Inisialisasi SMOTETomek
smt = SMOTETomek(random_state=42, n_jobs=-1)

# Lakukan resampling HANYA pada data latih
X_train_resampled, y_train_resampled = smt.fit_resample(X_train, y_train)

print("Proses resampling selesai.")
print("Distribusi kategori pada data latih (setelah resampling):")
print(sorted(Counter(y_train_resampled).items()))

# 6. Melatih Model Machine Learning pada Data Seimbang
print("\nMemulai proses pelatihan model RandomForest...")
# Buat model TANPA class_weight, karena data sudah seimbang secara fisik
model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    oob_score=True,
    n_jobs=-1
)

# Latih model menggunakan data hasil resampling
model.fit(X_train_resampled, y_train_resampled)
print("Pelatihan model selesai.")

# 7. Mengevaluasi Performa Model
print("\n--- Evaluasi Model (pada data uji asli) ---")
# Prediksi dilakukan pada data uji yang tidak pernah di-resampling
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Akurasi pada data uji: {accuracy:.4f}")
print(f"Out-of-Bag (OOB) Score: {model.oob_score_:.4f}")

print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred))


Memulai eksekusi skrip...
Dataset 'wine_quality.csv' berhasil dimuat.

Distribusi kategori asli:
quality_category
Middle    1319
Good       217
Bad         63
Name: count, dtype: int64

Dataset berhasil dibagi.
Distribusi kategori pada data latih (sebelum resampling):
[('Bad', 50), ('Good', 174), ('Middle', 1055)]

Menerapkan SMOTE-Tomek pada data latih... Ini mungkin butuh beberapa saat.
Proses resampling selesai.
Distribusi kategori pada data latih (setelah resampling):
[('Bad', 1041), ('Good', 1040), ('Middle', 1028)]

Memulai proses pelatihan model RandomForest...
Pelatihan model selesai.

--- Evaluasi Model (pada data uji asli) ---
Akurasi pada data uji: 0.8344
Out-of-Bag (OOB) Score: 0.9337

Laporan Klasifikasi:
              precision    recall  f1-score   support

         Bad       0.27      0.23      0.25        13
        Good       0.57      0.77      0.65        43
      Middle       0.92      0.88      0.90       264

    accuracy                           0.83       320


In [17]:

# 8. Menyimpan Model Baru
model_filename = 'wine_model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(model, file)

print(f"\nModel baru (dilatih dengan SMOTE-Tomek) telah disimpan sebagai '{model_filename}'.")


Model baru (dilatih dengan SMOTE-Tomek) telah disimpan sebagai 'wine_model.pkl'.
